# Fake News Classifier Using LSTM

In [1]:
# Import Dependencies
import tensorflow as tf 
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

In [2]:
import pandas as pd 

df = pd.read_csv("fake_news.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Check the null values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
# Drop Nan values
df = df.dropna()

In [5]:
# Get th Independent Features and dependent feature
X = df.drop("label" , axis=1)
y = df.label


In [6]:
X.shape , y.shape

((18285, 4), (18285,))

## One-Hot Representation

In [7]:
# Vocabulary size
voc_size=5000

In [8]:
messages = X.copy()

In [9]:
messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [10]:
messages["text"][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [11]:
messages.reset_index(inplace=True)

In [12]:
# import Dependencies for NLP task
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

1
14952
14953
14954
14955
14956
14957
14958
14959
14960
14961
14962
14963
14964
14965
14966
14967
14968
14969
14970
14971
14972
14973
14974
14975
14976
14977
14978
14979
14980
14981
14982
14983
14984
14985
14986
14987
14988
14989
14990
14991
14992
14993
14994
14995
14996
14997
14998
14999
15000
15001
15002
15003
15004
15005
15006
15007
15008
15009
15010
15011
15012
15013
15014
15015
15016
15017
15018
15019
15020
15021
15022
15023
15024
15025
15026
15027
15028
15029
15030
15031
15032
15033
15034
15035
15036
15037
15038
15039
15040
15041
15042
15043
15044
15045
15046
15047
15048
15049
15050
15051
15052
15053
15054
15055
15056
15057
15058
15059
15060
15061
15062
15063
15064
15065
15066
15067
15068
15069
15070
15071
15072
15073
15074
15075
15076
15077
15078
15079
15080
15081
15082
15083
15084
15085
15086
15087
15088
15089
15090
15091
15092
15093
15094
15095
15096
15097
15098
15099
15100
15101
15102
15103
15104
15105
15106
15107
15108
15109
15110
15111
15112
15113
15114
15115
15116
15117
15

In [15]:
corpus

chut new york time',
 'stranahan steve bannon nail media fight trump breitbart',
 'texa elector expect massiv corrupt relat elector colleg vote',
 'guilti power nullif counteract govern tyranni',
 'trillion new debt day',
 'tori councillor say homeless peopl elimin',
 'review hillbilli elegi tough love analysi poor back trump new york time',
 'crimin chief',
 'govern lie movi',
 'trump brexit defeat global anyway',
 'face congress sport offici begin confront sexual abus new york time',
 'airbnb end fight new york citi fine new york time',
 'big pharma martin shkreli suspend twitter breitbart',
 'next big tech corridor seattl vancouv planner hope new york time',
 'evid robot win race american job new york time',
 'see africa road new york time',
 'happen hip hop hillari goe dead broke brace',
 'bill maher high trump state free speech new era new york time',
 'l p g tour donald trump complic new york time',
 'legend art cashin trump presid new world order gold brexit great depress see pa

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

  4615,
  3674,
  3614,
  1511,
  2579,
  2304,
  3013,
  2608],
 [289, 2188, 1616, 2572, 4063],
 [2971, 4650, 2038, 4709, 2152, 459, 1567, 3461, 143],
 [360, 477, 4247, 2871, 2804, 3370],
 [3873, 3205, 723, 4854, 2203, 1567, 2237, 4697, 4558, 2534, 870, 2370],
 [4659, 1345, 4417, 3051],
 [581, 2347, 1363, 4653, 1223, 3525, 3440, 2728],
 [1617, 523, 659, 2516, 583, 175, 3883, 1567, 3461, 143],
 [922, 3102, 3243, 2688, 1718, 121, 133, 4594],
 [2923, 4988, 2923, 990, 3149, 4642],
 [4025, 4350, 4856, 3390, 2961, 1397, 4222, 1153, 1290, 1567, 3461, 143],
 [1681, 3699, 2835, 3323, 3668, 292, 576, 1330, 1244, 1567, 3461, 143],
 [101, 2892, 251, 1842, 807, 456, 2270, 389],
 [1336, 2599, 2835, 976, 10, 437, 4974, 2394, 1567, 3461, 143],
 [4940, 1443, 2545, 2038, 1757],
 [2134, 429, 3711, 3199, 2268, 3785, 2895, 4615, 3429, 1567, 3461, 143],
 [3105, 4960, 3342, 1277, 3783, 2362, 4051, 4636, 4266],
 [4125, 1135, 313, 3545, 4325, 514, 4659],
 [2162, 4854, 3675, 1794, 4659, 2815, 860, 1244],
 [260

## Embedding Representation

In [17]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr , padding="pre" , maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 4575  528  618]
 [   0    0    0 ... 3450 2864 2608]
 [   0    0    0 ... 2874 2002 2087]
 ...
 [   0    0    0 ... 1567 3461  143]
 [   0    0    0 ... 1775 1435 3061]
 [   0    0    0 ... 3832 1028 2862]]


In [18]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4130,
       4299,  576, 4946,  451, 3911, 3558, 4575,  528,  618])

In [19]:
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size , embedding_vector_features , input_length = sent_length))
model.add(LSTM(100)) # one LSTM layer with 100 neurons
model.add(Dense(1 , activation = "sigmoid"))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss="binary_crossentropy" , optimizer = "adam" , metrics=["accuracy"])

In [22]:
len(embedded_docs) , y.shape

(18285, (18285,))

In [23]:
import numpy as np 
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [24]:
X_final.shape , y.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final , y_final , test_size=0.33 , random_state=42)


## Training the model

In [26]:
# model.fit(X_train , y_train , validation_data=(X_test , y_test) , epochs=10 , batch_size = 64)

## Check the Metrics and Accuracy

In [27]:
# y_pred = model.predict_classes(X_test)

In [28]:
# y_test[:10]

In [29]:
# y_pred[:10]

In [30]:
# from sklearn.metrics import confusion_matrix

# confusion_matrix(y_test,y_pred)

In [31]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test,y_pred)

### If we are adding the Dropout layer 

In [32]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 72ms/step - loss: 0.3487 - accuracy: 0.8402 - val_loss: 0.2189 - val_accuracy: 0.9122
Epoch 2/10
192/192 [==============================] - 10s 55ms/step - loss: 0.1557 - accuracy: 0.9389 - val_loss: 0.1974 - val_accuracy: 0.9175
Epoch 3/10
192/192 [==============================] - 9s 47ms/step - loss: 0.1173 - accuracy: 0.9549 - val_loss: 0.2061 - val_accuracy: 0.9195
Epoch 4/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0891 - accuracy: 0.9675 - val_loss: 0.2202 - val_accuracy: 0.9185
Epoch 5/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0687 - accuracy: 0.9741 - val_loss: 0.3034 - val_accuracy: 0.9196
Epoch 6/10
192/192 [==============================] - 15s 77ms/step - loss: 0.0517 - accuracy: 0.9816 - val_loss: 0.2969 - val_accuracy: 0.9210
Epoch 7/10
192/192 [==============================] - 11s 57ms/step - loss: 0.0403 - accuracy: 0.9851 - val_loss: 0.3408 - val_accuracy: 

In [34]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [35]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[3123,  296],
       [ 188, 2428]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.91980115990058

Accuracy is much more better if we add the Dropout layer 

In [37]:
X_test

array([[   0,    0,    0, ..., 3372, 3370, 3361],
       [   0,    0,    0, ..., 1567, 3461,  143],
       [   0,    0,    0, ..., 1567, 3461,  143],
       ...,
       [   0,    0,    0, ..., 1567, 3461,  143],
       [   0,    0,    0, ...,    0,    0, 4659],
       [   0,    0,    0, ..., 2216,  367,  634]])

In [39]:
y_pred=model.predict_classes(X_test)

In [48]:
y_pred[0]

array([1])

In [49]:
if y_pred[0] == 0:
    print("This News is not Fake")

else:
    print("This News is Fake")

This News is Fake
